<a href="https://colab.research.google.com/github/NagarjunaD024/Datascience-LLMS/blob/main/src/Analyzing%20Structured%20data/Natural_Language_Query_Interface_Cypher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import argparse
import openai
import pandas as pd
import time
import re
from IPython.display import Markdown, display
from google.colab import files
import sqlite3

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from openai import OpenAI

from google.colab import userdata

client = OpenAI(api_key= userdata.get('secretName'))

In [ ]:
def create_prompt(question):
    """ Generate prompt to translate question into Cypher query.

    Args:
        question: question about data in natural language.

    Returns:
        prompt for question translation.
    """
    parts = []
    parts += ['Neo4j Database:']
    parts += ['Node labels: Movie, Person']
    parts += ['Relationship types: ACTED_IN, DIRECTED,']
    parts += ['FOLLOWS, PRODUCED, REVIEWED, WROTE']
    parts += ['Property keys: born, name, rating, released']
    parts += ['roles, summary, tagline, title']
    parts += [question]
    parts += ['Cypher Query:']
    return '\n'.join(parts)

In [ ]:
def call_llm(prompt):
    """ Query large language model and return answer.

    Args:
        prompt: input prompt for language model.

    Returns:
        Answer by language model.
    """
    for nr_retries in range(1, 4):
        try:
            response = client.chat.completions.create(
                model='gpt-4o',
                messages=[
                    {'role':'user', 'content':prompt}
                    ]
                )
            return response.choices[0].message.content
        except:
            time.sleep(nr_retries * 2)
    raise Exception('Cannot query OpenAI model!')

In [ ]:
prompt = create_prompt("How many movies keanu reeves acted are stored?")
print('--- Prompt ---')
print(prompt)

answer = call_llm(prompt)
print('--- Answer ---')
print(answer)

query = re.findall('```cypher(.*)```', answer, re.DOTALL)[0]
print('--- Query ---')
print(query)

--- Prompt ---
Neo4j Database:
Node labels: Movie, Person
Relationship types: ACTED_IN, DIRECTED,
FOLLOWS, PRODUCED, REVIEWED, WROTE
Property keys: born, name, rating, released
roles, summary, tagline, title
How many movies keanu reeves acted are stored?
Cypher Query:
--- Answer ---
To find out how many movies Keanu Reeves acted in that are stored in the Neo4j database, you can execute the following Cypher query:

```cypher
MATCH (keanu:Person {name: 'Keanu Reeves'})-[:ACTED_IN]->(m:Movie)
RETURN count(m) AS numberOfMovies;
```

This query matches a node with the label `Person` and the property `name` equal to 'Keanu Reeves', then follows the `ACTED_IN` relationship to connected `Movie` nodes, and finally counts and returns the number of such movies.
--- Query ---

MATCH (keanu:Person {name: 'Keanu Reeves'})-[:ACTED_IN]->(m:Movie)
RETURN count(m) AS numberOfMovies;

